In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from metadata_dict import (hca_keys, all_hca, tier1_metadata)

We should have the `filename` of the xlsx spreadsheet in the `path` with the correct `ext`

In the same path the cell_barcode csv file should exists that contains the following columns:
- cell_suspension.biomaterial_core.biomaterial_id
- cell_barcodes
- analysis_file

In [3]:
path = "."
file_name = "ImmuneLandscapeccRCC_metadata_30-01-2023"
ext = ".xlsx"

In [4]:
spreadsheet = pd.read_excel(path + "/dcp_spreadsheets/" + file_name + ext, sheet_name=None, skiprows= [0,1,2,4])

In [5]:
# Save all biomaterials and protocols from each tab as join keys

join_keys = {}
join_set = set()
for sheet in spreadsheet:
    join_keys[sheet] = []
    for key in spreadsheet[sheet].keys():
        if (key.endswith("biomaterial_id") or key.endswith("protocol_id")) and (not all(spreadsheet[sheet][key].isna())):
            join_keys[sheet].append(key)
            join_set.add(key)
    if not join_keys[sheet]:
        del join_keys[sheet]

In [6]:
# Save keys that have double pipe in order to merge metadata together for those keys

keys_with_double_pipe = []
for sheet in join_keys.keys():
    for keys in join_keys[sheet]:
        if any(spreadsheet[sheet][keys].str.contains("\\|\\|")):
            keys_with_double_pipe.append(keys)
keys_with_double_pipe

['enrichment_protocol.protocol_core.protocol_id',
 'dissociation_protocol.protocol_core.protocol_id',
 'cell_suspension.biomaterial_core.biomaterial_id']

In [7]:
# Manually order the spreadsheet tabs in the `ordered_sheet` list, in order to trace back the experimental design
# Most common experiemntal design is Analysis file -> CS -> Specimen -> Donor

list(join_keys.values())
# default ordering for simple experimental design & protocols
ordered_sheet = ['Analysis file', 'Cell suspension', 'Specimen from organism', 'Donor organism', 
                 'Sequencing protocol', 'Library preparation protocol', 'Dissociation protocol', 'Enrichment protocol', 'Collection protocol']

In [8]:
# filter out analysis files that will create duplicates i.e. barcodes and features for same CS and remove analysis_files that did not derive from CS
spreadsheet['Analysis file'] = spreadsheet['Analysis file'][spreadsheet['Analysis file']['analysis_file.file_core.content_description.ontology']\
        .isin(['data:3917', 'data:3112'])]\
    .dropna(subset='cell_suspension.biomaterial_core.biomaterial_id')

# If we have pooled CS for analysis_files, create a new entry for each CS of each file
analysis_pooledCS = spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id'].str.contains("\\|\\|")
if any(analysis_pooledCS):
    print(f"analysis files are pooled")
    spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id'] = \
        spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id']\
            .str.split('\\|\\|')\
            .apply(lambda x: x[0] if len(x) == 1 else x)
    spreadsheet['Analysis file'] = spreadsheet['Analysis file'].explode('cell_suspension.biomaterial_core.biomaterial_id')

# Use only files that are described as count matrices or gene expression matrices
bc = spreadsheet['Analysis file'][['cell_suspension.biomaterial_core.biomaterial_id','analysis_file.file_core.file_name']].drop_duplicates('cell_suspension.biomaterial_core.biomaterial_id')

analysis files are pooled


In [9]:
# If no analysis files available provide flatten metatadata at the sample level based on the CS from the sequence file tab
# create the backbone of the flatten metadata from the sequence file tab, removing all sequence file specific fields and removing duplicate entries 
if 'Analysis file' not in spreadsheet.keys():
    seq_specific_fields = ['sequence_file.file_core.file_name', 'sequence_file.file_core.format', 
                       'sequence_file.insdc_run_accessions', 'sequence_file.file_core.content_description.text', 
                       'sequence_file.file_core.content_description.ontology', 
                       'sequence_file.file_core.content_description.ontology_label', 'sequence_file.read_index',
                       'sequence_file.uuid', 'sequence_file.file_core.file_source', 'sequence_file.read_length', 
                       'process.uuid', 'process.insdc_experiment.insdc_experiment_accession', 'process.process_core.process_id', 
                       'sequencing_protocol.protocol_core.protocol_id', 'sequencing_protocol.uuid', 
                       'library_preparation_protocol.protocol_core.protocol_id', 'library_preparation_protocol.uuid', 
                       'sequence_file.lane_index']
    bc = spreadsheet['Sequence file'].dropna(axis=1, how= "all").drop(columns=seq_specific_fields).drop_duplicates()


In [10]:
# if we have pooled keys, edit the spreadsheet and add the pooled values as extra rows

def process_pooled_keys(sheet, bc, join_key):
    pooled_keys = [key for key in bc[join_key].dropna().unique().tolist() if "||" in key]
    print("We have pooled join keys", pooled_keys)
    for key in pooled_keys:
        unpooled_bool = sheet[join_key].isin(key.split("||"))
        summary_row = {}
        unique_values = {}
        # df.loc[unpooled_bool] = df.loc[unpooled_bool].astype('string')
        # unique_values = df.loc[unpooled_bool].apply(lambda col: col.unique()).to_dict()
        # unique_values = df.loc[unpooled_bool].apply(lambda col: col.unique()).to_dict('list')
        # unique_values = {col : list(contents.dropna().astype('str').unique()) for col, contents in df.loc[unpooled_bool].items()}
        for col, contents in sheet.loc[unpooled_bool].items():
            unique_values[col] = list(contents.dropna().astype('str').unique())
            if len(unique_values[col]) == 1:
                unique_values[col] = unique_values[col][0]
        for col, values in unique_values.items():
            # if 'nan' in values and len(values) > 1:
            #     unique_values[col] = values[values != 'nan']
            if isinstance(values, list) and len(values) > 1:
                summary_row[col] = '||'.join(values)
            else:
                summary_row[col] = values
        # summary_row = {col: '||'.join(values) if len(values) > 1 else values[0]
        #     for col, values in unique_values.items()}
        id_key = [name for name in summary_row.keys() if name.endswith("_id")]
        if len(id_key) > 1:
            print("Multiple id keys")
        summary_row[id_key[0]] = key
        return pd.concat([sheet, pd.DataFrame([summary_row])], ignore_index=True)


In [11]:
for sheet in ordered_sheet:
    if sheet not in spreadsheet.keys():
        continue
    keys_in_tab = [spreadsheet_keys for spreadsheet_keys in spreadsheet[sheet].keys() \
                        if spreadsheet_keys in hca_keys or \
                            spreadsheet_keys in join_set or \
                            spreadsheet_keys in tier1_metadata.values()]
    join_key = [key for key in keys_in_tab if key in bc.keys() and key in join_set]
    if len(join_key) > 1:
        print("Multiple join keys in " + sheet + ":\n\t" + "\n\t".join(join_key))
        if 'analysis_file.file_core.file_name' in join_key:
            join_key = 'analysis_file.file_core.file_name'
    else:
        join_key = join_key[0]

    keys_in_tab = [key for key in keys_in_tab if key not in bc.keys() or key == join_key]
    if join_key in keys_with_double_pipe and any(bc[join_key].str.contains("\\|\\|")):
        spreadsheet[sheet] = process_pooled_keys(spreadsheet[sheet], bc, join_key)

    print("Merging sheet " + sheet + " on key " + join_key, sep = "\t")
    bc = bc.merge(spreadsheet[sheet][keys_in_tab], on = join_key, how = "left")
    print("Shape is " + str(bc.shape))
bc

Merging sheet Analysis file on key cell_suspension.biomaterial_core.biomaterial_id
Shape is (29, 7)
Merging sheet Cell suspension on key cell_suspension.biomaterial_core.biomaterial_id
Shape is (29, 17)
Merging sheet Specimen from organism on key specimen_from_organism.biomaterial_core.biomaterial_id
Shape is (29, 28)
Merging sheet Donor organism on key donor_organism.biomaterial_core.biomaterial_id
Shape is (29, 48)
Merging sheet Sequencing protocol on key sequencing_protocol.protocol_core.protocol_id
Shape is (29, 55)
Merging sheet Library preparation protocol on key library_preparation_protocol.protocol_core.protocol_id
Shape is (29, 70)
We have pooled join keys ['enzymatic_dissociation||mechanical_dissociation']
Merging sheet Dissociation protocol on key dissociation_protocol.protocol_core.protocol_id
Shape is (29, 74)
We have pooled join keys ['live_cell_selection||size_selection']
Merging sheet Enrichment protocol on key enrichment_protocol.protocol_core.protocol_id
Shape is (29,

,cell_suspension.biomaterial_core.biomaterial_id,analysis_file.file_core.file_name,analysis_file.matrix_cell_count,analysis_file.genome_assembly_version,sequencing_protocol.protocol_core.protocol_id,analysis_protocol.protocol_core.protocol_id,library_preparation_protocol.protocol_core.protocol_id,cell_suspension.biomaterial_core.biomaterial_name,cell_suspension.biomaterial_core.biomaterial_description,cell_suspension.biomaterial_core.biosamples_accession,...,dissociation_protocol.method.ontology,dissociation_protocol.method.ontology_label,enrichment_protocol.method.text,enrichment_protocol.method.ontology,enrichment_protocol.method.ontology_label,enrichment_protocol.maximum_size,enrichment_protocol.markers,collection_protocol.method.text,collection_protocol.method.ontology,collection_protocol.method.ontology_label
0,IpiNivo_Complete_kidney_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from normal kidney tissue,Cell suspension from normal kidney tissue,SAMN18089850,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
1,IpiNivo_Complete_PBMC_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,PBMCs,PBMCs,SAMN18089851,...,NaN,NaN,density gradient centrifugation,EFO:0009112,density gradient centrifugation,NaN,NaN,blood draw,EFO:0009121,blood draw
2,NivoExposed_tumor_Near_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from tumor tissue (Near),Cell suspension from tumor tissue (Near),SAMN18089831,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
3,UT1_tumor_Center_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from tumor tissue (Center),Cell suspension from tumor tissue (Center),SAMN18089852,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
4,NivoExposed_tumor_Far_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from tumor tissue (Far),Cell suspension from tumor tissue (Far),SAMN18089833,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
5,UT2_tumor_Near_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from tumor tissue (Near),Cell suspension from tumor tissue (Near),SAMN18089828,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
6,UT2_kidney_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from normal kidney tissue,Cell suspension from normal kidney tissue,SAMN18089830,...,EFO:0009129||EFO:0009128,mechanical dissociation||enzymatic dissociation,cell size selection||FACS,EFO:0009337||EFO:0009108,cell size selection||fluorescence-activated ce...,100.0,DAPI-,surgical resection,EFO:0009744,surgical resection
7,IpiNivo_Mixed_kidney_CS,ccRCC_6pat_Seurat.rds,167283.0,GRCh37,10x_sequencing,matrix_generation,10x_library_prep,Cell suspension from normal kidney tissue,Cell suspension from normal kidney tissue,SAMN18089845,...,EFO

In [12]:
bc = bc.drop_duplicates(subset='cell_suspension.biomaterial_core.biomaterial_id')\
    .drop(columns=[col for col in bc.keys() if col.endswith('protocol_id')])

In [13]:
bc.drop_duplicates(subset='cell_suspension.biomaterial_core.biomaterial_id')\
    .to_csv(path + "/flat_dcp/"  + file_name + "_flat_biomaterial.csv", index = False)